In [1]:
import numpy as np
import pandas as pd

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.callbacks import EarlyStopping

from keras.datasets import imdb

2022-04-25 12:18:28.460092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-25 12:18:28.460139: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
n_words = 1000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=n_words)
print('Train seq: {}'.format(len(X_train)))
print('Test seq: {}'.format(len(X_train)))

Train seq: 25000
Test seq: 25000


In [3]:
# Pad sequences with max_len
max_len = 200
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [4]:
# Define network architecture and compile
model = Sequential()
model.add(Embedding(n_words, 50, input_length=max_len))
model.add(Dropout(0.2))

model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
#use simple RNN instead of GRU

# model.add(SimpleRNN(50, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(250, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',  optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           50000     
                                                                 
 dropout (Dropout)           (None, 200, 50)           0         
                                                                 
 gru (GRU)                   (None, 100)               45600     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 95,701
Trainable params: 95,701
Non-trainable params: 0
_________________________________________________________________


2022-04-25 12:18:32.160704: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-25 12:18:32.160749: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-25 12:18:32.160780: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Willy): /proc/driver/nvidia/version does not exist
2022-04-25 12:18:32.161036: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=2)]

In [6]:
batch_size = 512
n_epochs = 100

model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs, validation_split=0.2, callbacks=callbacks)

Epoch 1/100
40/40 [==============================] - 16s 369ms/step - loss: 0.7000 - accuracy: 0.5810 - val_loss: 0.6295 - val_accuracy: 0.6078
Epoch 2/100
40/40 [==============================] - 14s 356ms/step - loss: 0.6140 - accuracy: 0.6806 - val_loss: 0.5870 - val_accuracy: 0.7098
Epoch 3/100
40/40 [==============================] - 13s 330ms/step - loss: 0.4951 - accuracy: 0.7675 - val_loss: 0.4154 - val_accuracy: 0.8150
Epoch 4/100
40/40 [==============================] - 13s 320ms/step - loss: 0.3757 - accuracy: 0.8350 - val_loss: 0.3895 - val_accuracy: 0.8306
Epoch 5/100
40/40 [==============================] - 13s 335ms/step - loss: 0.3444 - accuracy: 0.8521 - val_loss: 0.4203 - val_accuracy: 0.8324
Epoch 6/100
40/40 [==============================] - 13s 336ms/step - loss: 0.3373 - accuracy: 0.8589 - val_loss: 0.3575 - val_accuracy: 0.8500
Epoch 7/100
40/40 [==============================] - 14s 340ms/step - loss: 0.3344 - accuracy: 0.8604 - val_loss: 0.3534 - val_accuracy:

In [7]:
print('Accuracy on test set: {}'.format(model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 8s 10ms/step - loss: 0.3441 - accuracy: 0.8515
Accuracy on test set: 0.8515200018882751


In [8]:
print(X_test)

[[  0   0   0 ...  14   6 717]
 [  2   2  45 ... 125   4   2]
 [  2 189   4 ...   9  57 975]
 ...
 [  0   0   0 ...  21 846   2]
 [  0   0   0 ...   2   7 470]
 [  0   0   0 ...  34   2   2]]
